# Velogames solver: Itzulia Women 2023


## Load libraries

In [24]:
using Velogames

## Load data

In [29]:
# pull the data on riders and their current points
rider_df = getvgriders("https://www.velogames.com/itzulia-women/2023/riders.php")
# rename cost to vgcost
rename!(rider_df, :cost => :vgcost)

Row,,rider,team,vgcost,points,riderkey,value
,String,String,String,Int64,Float64,String,Float64
1,,Demi Vollering,Team SD Worx,32,0.0,deegiillmnorv,0.0
2,,Annemiek Van Vleuten,Movistar Team,24,0.0,aaeeeeiklmnnnntuvv,0.0
3,,Liane Lippert,Movistar Team,20,0.0,aeeiillnpprt,0.0
4,,Katarzyna Niewiadoma,Canyon//SRAM Racing,18,0.0,aaaaadeiikmnnortwyz,0.0
5,,Évita Muzic,FDJ - SUEZ,18,0.0,aciimtuvz,0.0
6,,Pauliena Rooijakkers,Canyon//SRAM Racing,14,0.0,aaaeeiijkklnooprrsu,0.0
7,,Marta Cavalli,FDJ - SUEZ,14,0.0,aaaacillmrtv,0.0
8,,Marlen Reusser,Team SD Worx,14,0.0,aeeelmnrrrssu,0.0
9,,Krista Doebel-Hickok,EF Education-TIBCO-SVB,12,0.0,abcdeehiikkkloorst,0.0


In [30]:
mycols = [:gc_we, :oneday_we]
# getpcsranking for each col, filter the dataframe to only the points and riderkey columns, and rename the points column to the col name.
pcs_dfs = map(mycols) do col
    rename(getpcsranking(col), :points => col)[:, [col, :riderkey]]
end

# use reduce to join all dataframes in pcs_dfs on the riderkey column
pcs_df = reduce((x, y) -> outerjoin(x, y, on=:riderkey), pcs_dfs)

# join the velogames and pcs dataframes on the riderkey column
rider_df = leftjoin(rider_df, pcs_df, on=:riderkey)
rider_df = coalesce.(rider_df, 0)

# create calcpcsscore column which is the sum of All Rounder * gc, Sprinter * sprint, Climber * mountain, and Unclassed * overall
rider_df.calc_score = mean([rider_df.gc_we, rider_df.oneday_we]);

In [31]:
rider_df

Row,,rider,team,vgcost,points,riderkey,value,gc_we,oneday_we,calc_score
,String,String,String,Int64,Float64,String,Float64,Float64,Float64,Float64
1,,Demi Vollering,Team SD Worx,32,0.0,deegiillmnorv,0.0,876.0,1289.0,1082.5
2,,Annemiek Van Vleuten,Movistar Team,24,0.0,aaeeeeiklmnnnntuvv,0.0,1040.0,621.0,830.5
3,,Liane Lippert,Movistar Team,20,0.0,aeeiillnpprt,0.0,668.0,614.0,641.0
4,,Marlen Reusser,Team SD Worx,14,0.0,aeeelmnrrrssu,0.0,142.0,566.0,354.0
5,,Katarzyna Niewiadoma,Canyon//SRAM Racing,18,0.0,aaaaadeiikmnnortwyz,0.0,498.0,459.0,478.5
6,,Karlijn Swinkels,Team Jumbo-Visma,8,0.0,aeiijkkllnnrssw,0.0,243.0,338.0,290.5
7,,Loes Adegeest,FDJ - SUEZ,12,0.0,adeeeeglosst,0.0,217.0,326.0,271.5
8,,Soraya Paladin,Canyon//SRAM Racing,10,0.0,aaaadilnoprsy,0.0,180.0,325.0,252.5
9,,Tamara Dronova-Balabolina,Israel Premier Tech Roland,10,0.0,aaaaaaabbdillmnnooorrtv,0.0,268.0,297.0,282.5


## Build model

In [32]:
model_results = build_model_oneday(rider_df);

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 113 cols, 226 nonzeros
2 rows, 31 cols, 61 nonzeros
2 rows, 28 cols, 55 nonzeros
Objective function is integral with scale 2

Solving MIP model with:
   2 rows
   28 cols (26 binary, 2 integer, 0 implied int., 0 continuous)
   55 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   7581.5          -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   2960.142857     2805.5             5.51%        0      0      0         2     0.0s

14.3% inactive integer columns, restarting
Model after restart has 2 rows, 24 cols (23 bin., 1 int., 0 impl., 0 cont.), and 47 nonzeros


In [33]:
# number of riders selected
model_results.data |> sum

9.0

In [35]:
# total cost
rider_df.vgcost .* model_results.data |> sum

100.0

## Results

In [37]:
# selected riders
rider_df[!, :chosen] = model_results.data .> 0.5
chosen_team = filter(:chosen => ==(true), rider_df)
chosen_team[:, [:rider, :team, :chosen, :points, :calc_score, :vgcost]]

Row,rider,team,chosen,points,calc_score,vgcost
,String,String,Bool,Float64,Float64,Int64
1,Annemiek Van Vleuten,Movistar Team,true,0.0,830.5,24
2,Liane Lippert,Movistar Team,true,0.0,641.0,20
3,Karlijn Swinkels,Team Jumbo-Visma,true,0.0,290.5,8
4,Soraya Paladin,Canyon//SRAM Racing,true,0.0,252.5,10
5,Tamara Dronova-Balabolina,Israel Premier Tech Roland,true,0.0,282.5,10
6,Veronica Ewers,EF Education-TIBCO-SVB,true,0.0,401.0,10
7,Ella Wyllie,Lifeplus Wahoo,true,0.0,107.5,6
8,Georgia Williams,EF Education-TIBCO-SVB,true,0.0,140.0,8
9,Rosita Reijnhout,Team Jumbo-Visma,true,0.0,0.0,4
